In [7]:
import pickle
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Activation

## Load w2v model & Data

In [12]:
with open('../../data/w2v_pretrained_weights.pickle', 'rb') as handle:
    w2v_model = pickle.load(handle)
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

In [4]:
pretrained_weights = w2v_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


## Tokenization & Padding

In [10]:
MAX_NUM_WORDS = 20000

In [13]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_train['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_train['answer'])

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 30369
Length of longest sentence in input: 1688


In [14]:
x_train_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)
# x_train_pad = pad_sequences(input_integer_seq, maxlen=MAX_SEQ_LEN)
print("encoder_input_sequences.shape:", x_train_pad.shape)
print("encoder_input_sequences[172]:", x_train_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [  0   0   0 ... 135 183  86]


## Keras Model

In [16]:
def build_model(vocab_size,embedding_size,pretrained_weights):
    
           
    model = tf.keras.Sequential()
    
    inputs_placeholder = (Input(shape=(max_input_len,)))
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_size, 
                        weights=[pretrained_weights],
                        input_length=max_input_len)
#                         (inputs_placeholder)
                        )

    model.add(LSTM(units=embedding_size))
    model.add(Dense(units=vocab_size))
    model.add(Activation('softmax'))
    model.add(Dense(1))

    return model

In [17]:
def compile_model(model):
    mse = tf.keras.losses.MeanSquaredError()
    model.compile(loss=mse,
                  optimizer='adam',
                  metrics=['MeanSquaredError']
                  )
    return  model

In [18]:
def fit_model( x_train, y_train, model, batch_size,  epochs = 5):
    
    print('Train...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs)
    return model

In [23]:
with tf.device("/CPU:0"):
    model = build_model(vocab_size, embedding_size, pretrained_weights)
    model.summary()
    model = compile_model(model)
    model = fit_model(x_train_pad, y_train, model, batch_size=200, epochs=5)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1688, 300)         1131600   
_________________________________________________________________
lstm_4 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_8 (Dense)              (None, 3772)              1135372   
_________________________________________________________________
activation_4 (Activation)    (None, 3772)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 3773      
Total params: 2,991,945
Trainable params: 2,991,945
Non-trainable params: 0
_________________________________________________________________
Train...
Epoch 1/5


InvalidArgumentError:  indices[158,1645] = 9598 is not in [0, 3772)
	 [[node sequential_4/embedding_4/embedding_lookup (defined at <ipython-input-18-400ee7281b86>:6) ]] [Op:__inference_train_function_11337]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_4/embedding_4/embedding_lookup:
 sequential_4/embedding_4/embedding_lookup/10367 (defined at /usr/lib/python3.6/contextlib.py:81)

Function call stack:
train_function
